In [ ]:
import time

### Input data paths:

In [ ]:
#dataset_path = "../../Bio_Project/SmallDataset/reads_datasets/all_118309_1.fq"
#classifier_path = "../../Bio_Project/SmallDataset/classifiers_results/strex_centrifuge_118309.res"
#classifier_path = "../../Bio_Project/SmallDataset/classifiers_results/strex_kraken1_118309.res"
#classifier_path = "../../Bio_Project/SmallDataset/classifiers_results/strex_kraken2_118309.res"
#clusters_path =  "../../Bio_Project/results/paired/all_118309_1+RC.fasta.a16.t20.txt"
clusters_path =  "../../Bio_Project/results/LiME_binning_all_250000_1/all_250000_1+RC.fasta.a16.t20.txt"

dataset_path = "../../Bio_Project/SimDataset/reads_datasets/all_250000_1.fq"
#classifier_path = "../../Bio_Project/SimDataset/classifiers_results/strex_centrifuge_250000.res"
classifier_path = "../../Bio_Project/SimDataset/classifiers_results/strex_kraken1_250000.res"
#classifier_path = "../../Bio_Project/SimDataset/classifiers_results/strex_kraken2_250000.res"
#classifier_path = "../../Bio_Project/SimDataset/classifiers_results/strex_clark_genus_250000.res"
#classifier_path = "../../Bio_Project/SimDataset/classifiers_results/strex_clark_species_250000.res"


#### Load dataset function 
- If dataset is in fasta format then dataset_format parameter equals TRUE.
- If dataset is in fastq format then dataset_format equals FALSE.


In [ ]:
def load_dataset(path, datset_format : bool):

    dataset = open(path, "r")
    dataset_lines = []

    if (datset_format):
        divisor = 2
    else:
        divisor = 4

    index = 0
    for line in dataset:
        if (index%divisor==0):
            read_id = line.split()[0]
            dataset_lines.append(read_id[1: len(read_id)-2])
        index = index + 1

    dataset.close()

    return dataset_lines

#### Load classifier result function

In [ ]:
def load_classifier_result(path):

    classification = open(path, 'r')
    classifier_results = []

    for line in classification:
        col = []
        for j in range(0, len(line.split())):
            col.append(line.split()[j])
        #read_id = line.split()[0]
        #class_id = line.split()[1]
        #col.append(read_id)
        #col.append(class_id)
        classifier_results.append(col)

    classification.close()

    return classifier_results 

In [ ]:
def load_clusters_result(path):

    clusters = open(path, "r")
    clusters_list = []

    for group in clusters:
        clusters_list.append(int(group))

    clusters.close()

    return clusters_list

In [ ]:
def get_inverted_index(clusters, classified_reads):

    num_clusters = -1
    for i in range (0, len(clusters_list)):
        if (clusters_list[i] >= num_clusters):
            num_clusters = clusters_list[i]
    
    num_clusters = num_clusters + 1
    num_reads = len(clusters)
    inverted_index = []
    
    for i in range(0, num_clusters):
        inverted_index.append([])
   
    for i in range(0, num_reads):
        inverted_index[clusters[i]].append(classified_reads[i])

    return inverted_index

### Main function:

In [18]:
dataset_lines = load_dataset(dataset_path, False)
clusters_list = load_clusters_result(clusters_path)
classifier_results = load_classifier_result(classifier_path)

if (len(dataset_lines) != len(classifier_results)):
    print("Different length!")
    
i = 0
j = 0

all_read_label_list = []
count_mis = 0
count_match = 0
while(i < len(dataset_lines) and j < len(classifier_results)):
    
    if (dataset_lines[i] == classifier_results[j][0]):
        all_read_label_list.append(classifier_results[j])
        count_match = count_match + 1
        i = i + 1
        j = j + 1
    else:
        count_mis = count_mis + 1
        all_read_label_list.append([dataset_lines[i], '0'])
        i = i + 1

while (i != len(dataset_lines)):
    all_read_label_list.append([dataset_lines[i], '0'])
    i = i + 1
    count_mis = count_mis + 1

print("mismatches: ", count_mis)
print("matches: ", count_match)

if (count_match + count_mis == len(dataset_lines)):
    print("ok")

print(len(all_read_label_list))


mismatches:  624560
matches:  2500440
ok
3125000


In [19]:
inverted_index = get_inverted_index(clusters_list, all_read_label_list)

print(len(inverted_index))

735204


In [20]:
print(inverted_index[0])
print(inverted_index[500])

[['taxid_1042876.1', '76759']]
[['taxid_1042876.516', '76759'], ['taxid_1042876.41020', '76759'], ['taxid_1042876.62341', '76759']]


In [23]:
def frequency_search(cluster):
    
    label_dict= {}
       
    for element in cluster:
        element_label = element[1]
        if element_label in list(label_dict):
            label_dict[element_label] = label_dict[element_label] + 1
        else:
            # if it does not exist, it's automatically created
            label_dict[element_label] = 1
        
    return label_dict

In [24]:
label_dict_cluster_500 = frequency_search(inverted_index[500])

In [27]:
for label, frequency in label_dict_cluster_500.items():
    print(label, frequency)

1
76759 3


In [29]:
all_dict = []

for i in range (0, len(inverted_index)):
    label_dict_cluster = frequency_search(inverted_index[i])
    all_dict.append(label_dict_cluster)
    
print(len(all_dict))
    

735204
